<h1> Материалы курса "Анализ данных в инфромационной безопасности" </h1>

<h2> Практическое занятие № 6.1 </h2>

<h2> Тема: "Обнаружение уязвимостей в коде" </h2>

<h2>Автор:</h2>

<p>Русинова Залина</p>
<p>Младший исследователь исследовательского центра UDV Group</p>
<p>zalina.rusinova@udv.group</p>

## Оглавление <a name='toc'></a>
<ol>
<li><a href='#load'> Загрузка и подготовка данных</a></li>
<li><a href='#bert'>СodeBert</a></li>
</ol>

In [16]:
from tqdm import tqdm, trange
import multiprocessing

from transformers import (WEIGHTS_NAME, AdamW, get_linear_schedule_with_warmup,
                          RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer)
from datasets import Dataset
from transformers import RobertaTokenizer, RobertaForMaskedLM, pipeline
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

import torch

!pip install evaluate
import evaluate
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

### 1. Загрузка и подготовка данных <a name='load'></a>

Набор данных
[SARD](https://www.kaggle.com/datasets/cerengen/securec-c-source-code-for-vulnerabilty-detection/)

Software Assurance Reference Dataset (SARD) - представляет собой растущую коллекцию тестовых программ с документированными уязвимостями в коде. Тестовые примеры варьируются от небольших синтетических программ до крупных приложений. Программы написаны на C, C++, Java, PHP и C# и охватывают более 150 классов уязвимостей. 
- Коллекция из более чем 450000 примеров уязвимостей;
- Охватывает более 150 классов Common Weakness Enumeration (CWE)

In [2]:
pd.set_option('max_colwidth', 400)
df = pd.read_csv('multi_sard_all_data_marked.csv')
df.head()

,code,CWE-Type,DataType
0,"\n \n \n #include ""IncludeMarker""\n \n #include <IncludeMarker>\n \n #ifdef _WIN32\n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #pragma comment(lib, ""ws2_32"") \n #define DefineMarker closesocket\n #else \n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #define DefineMar...",CWE114,SARD
1,"\n \n \n #include ""IncludeMarker""\n \n #include <IncludeMarker>\n \n #ifdef _WIN32\n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #pragma comment(lib, ""ws2_32"") \n #define DefineMarker closesocket\n #else \n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #define DefineMar...",CWE114,SARD
2,"\n \n \n #include ""IncludeMarker""\n \n #include <IncludeMarker>\n \n #ifdef _WIN32\n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #pragma comment(lib, ""ws2_32"") \n #define DefineMarker closesocket\n #else \n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #define DefineMar...",CWE114,SARD
3,"\n \n \n #include ""IncludeMarker""\n \n #include <IncludeMarker>\n \n #ifdef _WIN32\n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #pragma comment(lib, ""ws2_32"") \n #define DefineMarker closesocket\n #else \n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #define DefineMar...",CWE114,SARD
4,"\n \n \n #include ""IncludeMarker""\n \n #include <IncludeMarker>\n \n #ifdef _WIN32\n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #pragma comment(lib, ""ws2_32"") \n #define DefineMarker closesocket\n #else \n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #define DefineMar...",CWE114,SARD


In [3]:
unique = len(df['CWE-Type'].unique())
print(f'Количество записей в датасете: {len(df)}')
print(f'Типов CWE: {unique}')

Количество записей в датасете: 52802
Типов CWE: 109


In [4]:
id2label = dict()
label2id = dict()
ind = 0
for i in df['CWE-Type'].unique():
    id2label[ind] = i
    label2id[i] = ind
    ind+=1

In [5]:
df['label']=df['CWE-Type'].map(label2id)
df.head()

,code,CWE-Type,DataType,label
0,"\n \n \n #include ""IncludeMarker""\n \n #include <IncludeMarker>\n \n #ifdef _WIN32\n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #pragma comment(lib, ""ws2_32"") \n #define DefineMarker closesocket\n #else \n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #define DefineMar...",CWE114,SARD,0
1,"\n \n \n #include ""IncludeMarker""\n \n #include <IncludeMarker>\n \n #ifdef _WIN32\n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #pragma comment(lib, ""ws2_32"") \n #define DefineMarker closesocket\n #else \n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #define DefineMar...",CWE114,SARD,0
2,"\n \n \n #include ""IncludeMarker""\n \n #include <IncludeMarker>\n \n #ifdef _WIN32\n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #pragma comment(lib, ""ws2_32"") \n #define DefineMarker closesocket\n #else \n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #define DefineMar...",CWE114,SARD,0
3,"\n \n \n #include ""IncludeMarker""\n \n #include <IncludeMarker>\n \n #ifdef _WIN32\n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #pragma comment(lib, ""ws2_32"") \n #define DefineMarker closesocket\n #else \n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #define DefineMar...",CWE114,SARD,0
4,"\n \n \n #include ""IncludeMarker""\n \n #include <IncludeMarker>\n \n #ifdef _WIN32\n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #pragma comment(lib, ""ws2_32"") \n #define DefineMarker closesocket\n #else \n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #include <IncludeMarker>\n #define DefineMar...",CWE114,SARD,0


Разобъем датасет на тренировочную и отложенную выборку

In [6]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

### 2. Модель CodeBert <a name='bert'></a>

Для загрузки данных мы будем использовать библиотеку [Datasets](https://huggingface.co/docs/datasets/index). 

**Datasets** - это библиотека для простого доступа к наборам данных и совместного их использования для задач аудио, компьютерного зрения и обработки естественного языка (NLP).

Позволяет загружать наборы данных и использовать встроенные методы обработки данных для подготовки датасетов для использования в обучении нейросетевых моделей. Библиотека поддерживает формат Apache Arrow и обрабатывает большие наборы данных без каких-либо ограничений памяти для оптимальной скорости и эффективности. Также библиотека интегрировна с Hugging Face Hub, что позволяет легко загружать набор данных и делиться им с сообществом.

In [7]:
dataset = {}
dataset['text'] = list(df_train['code'])
dataset['label'] = list(df_train['label'])
ds = Dataset.from_dict(dataset)
ds = ds.train_test_split(test_size=0.1)

Токенизируем входные последовательности с текстом программ с помощью обученного токенизатора Сodebert-base-mlm

In [8]:
tokenizer = RobertaTokenizer.from_pretrained('microsoft/codebert-base-mlm')

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/504 [00:00<?, ?B/s]

In [9]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_dataset = ds.map(preprocess_function, batched=True)

  0%|          | 0/39 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

**Data Collator**

Data Collators - это объекты, которые будут формировать батчи (порции данных), используя список элементов набора данных в качестве входных данных. Эти элементы того же типа, что и элементы train_dataset или eval_dataset.

Чтобы иметь возможность создавать батчи, сборщики данных могут применять некоторую обработку (например, padding). 

In [10]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

**Evaluate**
Библиотека для простой оценки моделей машинного обучения и наборов данных.

С помощью одной строки кода библиотека дает доступ к десяткам методов оценки для различных областей (NLP, компьютерное зрение, обучение с подкреплением и др.)

In [11]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

[CodeBert](https://github.com/microsoft/CodeBERT)

Предварительно обученная модель, взятая из статьи исследовательского подразделения Microsoft. Для обучения этой модели использовался набор данных CodeSearchNet, но вместо генерирования комментариев он использовался для обучения модели на основе RoBERTa удобному для восприятия представлению кода и естественного языка.

In [12]:
model = AutoModelForSequenceClassification.from_pretrained(
    "microsoft/codebert-base-mlm", num_labels=109, id2label=id2label, label2id=label2id
)

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base-mlm and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Класс **Trainer** предоставляет API для полнофункционального обучения в PyTorch для большинства стандартных вариантов использования. 

Перед созданием экземпляра Trainer необходимо создать аргументы для обучения, чтобы получить доступ ко всем точкам настройки во время обучения.

API поддерживает распределенное обучение на нескольких графических процессорах, смешанную точность с помощью NVIDIA Apex и собственный ускоритель для PyTorch.

Trainer содержит базовый цикл обучения, который поддерживает вышеуказанные функции. Чтобы внедрить пользовательские функции, можно переопределить следующие методы:
- get_train_dataloader — Creates the training DataLoader.
- get_eval_dataloader — Creates the evaluation DataLoader.
- get_test_dataloader — Creates the test DataLoader.
- log — Logs information on the various objects watching training.
- create_optimizer_and_scheduler — Sets up the optimizer and learning rate scheduler if they were not passed at - init. Note, that you can also subclass or override the create_optimizer and create_scheduler methods separately.
- create_optimizer — Sets up the optimizer if it wasn’t passed at init.
- create_scheduler — Sets up the learning rate scheduler if it wasn’t passed at init.
- compute_loss - Computes the loss on a batch of training inputs.
- training_step — Performs a training step.
- prediction_step — Performs an evaluation/test step.
- evaluate — Runs an evaluation loop and returns metrics.
- predict — Returns predictions (with metrics if labels are available) on a test set.

In [13]:
training_args = TrainingArguments(
    output_dir="trained_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="wandb"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy
1,0.669600,0.546267,0.864379


TrainOutput(global_step=2376, training_loss=1.275373895561655, metrics={'train_runtime': 2249.2394, 'train_samples_per_second': 16.902, 'train_steps_per_second': 1.056, 'total_flos': 1.0012039313227776e+16, 'train_loss': 1.275373895561655, 'epoch': 1.0})

### Тестирование обученной модели

In [18]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

preds = []
for i in df_test['code'].values:
    with torch.no_grad():
        inputs = tokenizer(i, return_tensors="pt",  truncation=True).to(device)
        logits = model(**inputs).logits
        predicted_class_id = logits.argmax().item()
        preds.append(predicted_class_id)

In [20]:
y_true = [id2label[i] for i in df_test['label'].values]
y_pred = [id2label[i] for i in preds]

In [21]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

      CWE114       0.98      1.00      0.99       131
      CWE121       0.84      0.97      0.90      1154
      CWE122       1.00      0.83      0.90       734
      CWE123       1.00      1.00      1.00        30
      CWE124       0.77      0.65      0.71       351
      CWE126       0.93      0.79      0.85       255
      CWE127       0.80      0.77      0.78       376
      CWE134       1.00      1.00      1.00       649
       CWE15       0.00      0.00      0.00         9
      CWE176       0.00      0.00      0.00        10
      CWE188       0.00      0.00      0.00         9
      CWE190       0.93      0.89      0.91       970
      CWE191       0.83      0.95      0.89       739
      CWE194       0.95      1.00      0.97       245
      CWE195       0.63      0.98      0.77       259
      CWE196       0.00      0.00      0.00         3
      CWE197       0.96      0.71      0.81       194
      CWE222       0.00    

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
